<a href="https://colab.research.google.com/github/Kaif10/Doing-Stuff-with-ChatGPT./blob/main/Therapy_with_Chatgpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Therapy bot with Chatgpt api


Project

1. Audio of a person expressing how their life is going.
2. Converting audio to text for LLMs
3. Detecting the sentiment from that converted text
4. Putting that detected sentiment in Chatgpt and asking it to suggest therapy based on that.

In [1]:
!pip install python-dotenv
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.3 MB/s eta 0:00:00


In [2]:
#api key
import openai
import os


from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

#a.txt file with api key
openai.api_key_path= '/content/b.txt'

Install the required libraries

In [3]:
!pip install pydub
!pip install google-cloud-speech
!pip install transformers
!pip install SpeechRecognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.7/273.7 kB 4.4 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 17.4 MB/s eta 0:00:00


In [4]:
#import the libraries
from pydub import AudioSegment
from pydub.playback import play
from transformers import pipeline
import speech_recognition as sr
from google.cloud import speech



In [5]:
# Initialize Sentiment Analysis model
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
sentiment_classifier = pipeline("sentiment-analysis", model=model_name)

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [6]:
# Function to get the sample rate of the audio
def get_sample_rate(audio_path):
    audio = AudioSegment.from_file(audio_path)
    return audio.frame_rate



### Function to convert audio to text using speech_recognition library


In [7]:

def convert_audio_to_text(audio_path):
    # Load audio file using pydub
    audio = AudioSegment.from_file(audio_path)

    # Check if the audio is in a format compatible with the speech_recognition library
    if audio.sample_width != 2 or audio.frame_rate != 16000:
        # Convert the audio to the required format (16-bit PCM WAV with a sample rate of 16 kHz)
        audio = audio.set_frame_rate(16000).set_sample_width(2)

    # Save the audio to a temporary WAV file for speech recognition
    temp_wav_path = "/tmp/audio_temp.wav"
    audio.export(temp_wav_path, format="wav")

    # Initialize the recognizer
    recognizer = sr.Recognizer()

    # Convert audio to text using the recognizer
    with sr.AudioFile(temp_wav_path) as source:
        audio_data = recognizer.record(source)

    try:
        # Recognize speech using Google Web Speech API
        text = recognizer.recognize_google(audio_data)
        return text
    except sr.UnknownValueError:
        print("Google Web Speech API could not understand the audio.")
        return ""
    except sr.RequestError as e:
        print("Could not request results from Google Web Speech API; {0}".format(e))
        return ""

In [8]:
# Function to load audio file
def load_audio(audio_path):
    audio = AudioSegment.from_file(audio_path)
    return audio


# Function to detect sentiment from text with sentiment_classifier of transformers
def detect_sentiment(text):
    result = sentiment_classifier(text)
    sentiment = result[0]['label']
    return sentiment



.

### Chatgpt api suggesting us the therapy

In [9]:

def suggest_therapy(emotion):
    # Generate therapy suggestion using ChatGPT API
    response = get_completion(prompt= f""" I'm feeling {emotion}. What therapy would you recommend? """)


    return response

##Important

### Chatgpt 3.5 turbo model

In [10]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]


#Use the helper function
#response = get_completion(prompt)
#print(response)

### Main function

In [11]:
def main():
    # Provide the path to the audio file (person describing sadness and stress in this audio)
    audio_path = "/content/Recording.wav"

    # Convert audio to text
    text = convert_audio_to_text(audio_path)

    # Detect sentiment from text
    sentiment = detect_sentiment(text)

    # Suggest therapy based on sentiment
    therapy_suggestion = suggest_therapy(sentiment)

    # Print the therapy suggestion
    print("Therapy Suggestion:", therapy_suggestion)

# Run the application
if __name__ == '__main__':
    main()

Therapy Suggestion: I'm not a therapist, but there are several types of therapy that may be helpful for managing negative emotions. Here are a few options you could consider:

1. Cognitive Behavioral Therapy (CBT): This type of therapy focuses on identifying and changing negative thought patterns and behaviors that contribute to negative emotions.

2. Dialectical Behavior Therapy (DBT): DBT is often used to treat individuals with intense emotions. It combines elements of CBT with mindfulness techniques to help regulate emotions and improve coping skills.

3. Acceptance and Commitment Therapy (ACT): ACT aims to help individuals accept their negative emotions and thoughts while committing to actions that align with their values.

4. Mindfulness-Based Stress Reduction (MBSR): This therapy incorporates mindfulness meditation and other techniques to help individuals become more aware of their thoughts and emotions, and learn to respond to them in a non-judgmental way.

It's important to con